# BayesC0
##Rohan L. Fernando, Hao Cheng and Dorian Garrick
## November 2015

*You are free to use this code in your work, but please acknowledge the JWAS package and website until the associated paper, "JWAS: Julia Implementation of Whole-Genome Analyses Software", is published.*

# Simulating Genotypes and Phenotypes

In [2]:
using(Distributions)


In [3]:
nObs     = 100
nMarkers = 1000
X = sample([0,1,2],(nObs,nMarkers))
α  = randn(nMarkers)
a  = X*α
stdGen = std(a)
a = a/stdGen
y = a + randn(nObs)
saveAlpha = α
nothing

####Centering Genotype Covariates

In [4]:
meanXCols = mean(X,1)
X = X - ones(nObs,1)*meanXCols;

# Priors

In [5]:
seed            =   10    # set the seed for the random number generator
chainLength     =  2000   # number of iterations
probFixed       =    0    # parameter "pi" the probability SNP effect is zero
dfEffectVar     =    4    # hyper parameter (degrees of freedom) for locus effect variance          
nuRes           =    4    # hyper parameter (degrees of freedom) for residual variance
varGenotypic    =    1    # used to derive hyper parameter (scale) for locus effect variance 
varResidual     =    1    # used to derive hyper parameter (scale) for locus effect variance 
scaleVar        =   varGenotypic*(dfEffectVar-2)/dfEffectVar    # scale factor for locus effects
scaleRes        =   varResidual*(nuRes-2)/nuRes                 # scale factor for residual variance
nothing

## Function for Sampling Marker Effects

First we make an array, *xArray*, that contains the columns of $\mathbf{X}$. This is done without duplicating each column of $\mathbf{X}$.

In [6]:
function get_column(X,nRows,j)
    indx = 1 + (j-1)*nRows
    ptr = pointer(X,indx)
    pointer_to_array(ptr,nRows)
end

get_column (generic function with 1 method)

In [7]:
xpx = [(X[:,i]'X[:,i])[1]::Float64 for i=1:nMarkers]
xArray = Array(Array{Float64,1},nMarkers)
for i=1:nMarkers
    xArray[i] = get_column(X,nObs,i)
end

In [7]:
typeof(xArray[1])

Array{Float64,1}

### Computing the adjusted right-hand-side efficiently

We want to compute:
$$
rhs = \mathbf{X}_j'(\mathbf{y}_{corr} + \mathbf{X}_j\mathbf{\alpha}_j),
$$
where $\mathbf{y}_{corr} = \mathbf{y} - \mathbf{X}\mathbf{\alpha}.$ This is more efficiently obtained as
$$
rhs = \mathbf{X}_j'\mathbf{y}_{corr} + \mathbf{X}_j'\mathbf{X}_j\mathbf{\alpha}_j,
$$
using the diagonals of $\mathbf{X}'\mathbf{X}$ that have already been computed (line 4 of the function below).

In [8]:
function sampleEffects!(nMarkers,xArray,xpx,yCorr,α,meanAlpha,vare,varEffects)
    nObs = size(X,1)
    for j=1:nMarkers
        rhs::Float64      = dot(xArray[j],yCorr) + xpx[j]*α[j]   
        lhs::Float64      = xpx[j] + vare/varEffects
        invLhs::Float64   = 1.0/lhs
        mean::Float64     = invLhs*rhs
        oldAlpha::Float64 = α[j] 
        α[j] = mean + randn()*sqrt(invLhs*vare)
        BLAS.axpy!(oldAlpha-α[j],xArray[j],yCorr)  
    end
    nothing
end

sampleEffects! (generic function with 1 method)

## Function for BayesC0

The intercept is sampled first and the sampleEffects! function is called to sample the marker effects.

In [10]:
chi1=Chisq(nObs+nuRes)
chi2=Chisq(dfEffectVar+nMarkers)

function BayesC0!(numIter,nMarkers,X,xpx,yCorr,mu,meanMu,α,meanAlpha,vare,varEffects)
    for i=1:numIter
        # sample residula variance
        vare = (dot(yCorr,yCorr)+nuRes*scaleRes)/rand(chi1)
        
        # sample intercept
        yCorr  = yCorr+mu
        rhs    = sum(yCorr)                   
        invLhs = 1.0/(nObs)               
        mean   = rhs*invLhs   
        mu     = mean + randn()*sqrt(invLhs*vare)
        yCorr  = yCorr - mu                    
        meanMu = meanMu + (mu - meanMu)/i
        
        # sample effects
        sampleEffects!(nMarkers,xArray,xpx,yCorr,α,meanAlpha,vare,varEffects)
        meanAlpha = meanAlpha + (α - meanAlpha)/i
        
        #sameple locus effect variance
        varEffects = (scaleVar*dfEffectVar + dot(α,α))/rand(chi2)  
               
        if (i%1000)==0
            yhat = meanMu+X*meanAlpha
            resCorr = cor(a,yhat)
            println("Correlation of between true and predicted breeding value: ", resCorr)
        end
    end
end    

BayesC0! (generic function with 1 method)

##Run BayesC0

In [15]:
meanMu    = 0
meanAlpha = zeros(nMarkers)   

#initial valus
vare = 1
varEffects = 1
mu = mean(y)
yCorr = y - mu
alpha = fill(0.0,nMarkers)

#run it
@time BayesC0!(chainLength,nMarkers,X,xpx,yCorr,mu,meanMu,alpha,meanAlpha,vare,varEffects)

Correlation of between true and predicted breeding value: 0.5966097333301084
Correlation of between true and predicted breeding value: 0.596851172103314
  0.229751 seconds (64.12 k allocations: 50.504 MB, 1.63% gc time)


##Compare Runtime with R Implementation

In [12]:
# this not not working on Mac OS X, El Capitan
# ;Rscript RBayesC0/BayesC0.R

   user  system elapsed 
 46.792   1.248  48.058 


In [16]:
48.34/.23

210.17391304347828

In [14]:
;cat RBayesC0/BayesC0.R

#   This code is for illustrative purposes and not efficient for large problems
#   Real life data analysis (using the same file formats) is available at 
#   bigs.ansci.iastate.edu/login.html based on GenSel cpp software implementation
# 
#               Rohan Fernando      (rohan@iastate.edu)
#               Dorian Garrick      (dorian@iastate.edu) 
#               copyright August 2012

# Parameters
setwd("RBayesC0")
seed            =   10    # set the seed for the random number generator
chainLength     =  2000    # number of iterations
dfEffectVar     =    4    # hyper parameter (degrees of freedom) for locus effect variance          
nuRes           =    4    # hyper parameter (degrees of freedom) for residual variance
varGenotypic    =    1    # used to derive hyper parameter (scale) for locus effect variance 
varResidual     =    1    # used to derive hyper parameter (scale) for residual variance 
windowSize      =   10    # number of consecutive markers in a genomic window
out